In [2]:
from __future__ import division

import json
import pickle
import pandas as pd
import numpy as np

In [16]:
with open('data/general/matches_2017_2018_v1.json') as matches_json:
    matches_json = pd.read_json(matches_json)
    
matches = [60565]


no_team = 0
home_team = 0
away_team = 0
match_count = 1



data_height = 6000 # number of seconds in per match
data_width = 128 # number of features for fector


for id in matches:
    with open('data/match_' + str(id) + '/match_data_' + str(id) + '.json', 'r') as file:
        team = json.load(file)
        
    
        
    # defining 3 distinct id in game
    referee_id = 0
    home_team_id = int(team[1]['teamId'])
    away_team_id = int(team[2]['teamId'])
    

    # home and away teams' pos_avrg for 5 min, pos at the time and count of how many time they own the ball
    home_team_pos_avrg = {'xpos' : {}, 'ypos' : {}}
    away_team_pos_avrg = {'xpos' : {}, 'ypos' : {}}
    
    home_team_pos_t = {'xpos' : {}, 'ypos' : {}}
    away_team_pos_t = {'xpos' : {}, 'ypos' : {}}      
    
    home_team_pos_count = {}
    away_team_pos_count = {}
    
    # referee pos at the time
    referee_pos_t = {}
    referee_pos_t['xpos'] = 0
    referee_pos_t['ypos'] = 0
    
    
    with open('data/match_' + str(id) + '/roster_data_' + str(id) + '.json', 'r') as file:
        match_squad = json.load(file)
        
    for roaster in match_squad:
        if roaster['teamId'] == home_team_id:
            home_team_pos_avrg['xpos'][roaster['jerseyNumber']] = 0
            home_team_pos_avrg['ypos'][roaster['jerseyNumber']] = 0
            home_team_pos_count[roaster['jerseyNumber']] = 0
            
        elif roaster['teamId'] == away_team_id:
            away_team_pos_avrg['xpos'][roaster['jerseyNumber']] = 0
            away_team_pos_avrg['ypos'][roaster['jerseyNumber']] = 0
            away_team_pos_count[roaster['jerseyNumber']] = 0
            
            
    # create array of x input data for every match
    x_intput = np.ndarray(shape=(data_height, data_width), dtype=np.float)        
            
    with open('data/match_' + str(id) + '/per_sec_data_' + str(id) + '.json', 'r') as file:
        data_persec = json.load(file)
        
        
    # create first average positon data for assigning player position
    for t in data_persec:
        
        i = int(t['minute'])*60 + int(t['second'])
        x_pos = t['xpos']
        y_pos = t['ypos']
        team_id_t = int(t['teamId'])
        jersey_number_t = int(t['jerseyNumber'])
        
        if i==300:
            break
        
        if team_id_t == home_team_id:
            home_team_pos_avrg['xpos'][jersey_number_t] += x_pos
            home_team_pos_avrg['ypos'][jersey_number_t] += y_pos
            if x_pos!=0 or y_pos!=0:
                home_team_pos_count[jersey_number_t] +=1                

        elif team_id_t == away_team_id:
            away_team_pos_avrg['xpos'][jersey_number_t] += x_pos
            away_team_pos_avrg['ypos'][jersey_number_t] += y_pos
            if x_pos!=0 or y_pos!=0:
                away_team_pos_count[jersey_number_t] +=1
            
        
    
           
    tmp_t = 0 # temporary t 
        
    for d in data_persec:
        
        
        i = int(d['minute'])*60 + int(d['second'])
        x_pos = d['xpos']
        y_pos = d['ypos']
        team_id_t = int(d['teamId'])
        jersey_number_t = int(d['jerseyNumber'])
        
        
        # assign player to groups then clear pos_avr values in each 5 mins
        if (i+1)%300 == 0 and tmp_t != i+1 and i>299:
            
            tmp_t = i+1
            
            for key, value in home_team_pos_avrg['xpos'].items():
                try:
                    home_team_pos_avrg['xpos'][key] /= home_team_pos_count[key]
                except ZeroDivisionError:
                    home_team_pos_avrg['xpos'][key] = 0
                
            for key, value in home_team_pos_avrg['ypos'].items():
                try:
                    home_team_pos_avrg['ypos'][key] /= home_team_pos_count[key]
                except ZeroDivisionError:
                    home_team_pos_avrg['ypos'][key] = 0
                    
            
            # away team
            for key, value in away_team_pos_avrg['xpos'].items():
                try:
                    away_team_pos_avrg['xpos'][key] /= away_team_pos_count[key]
                except ZeroDivisionError:
                    away_team_pos_avrg['xpos'][key] = 0
                
            for key, value in away_team_pos_avrg['ypos'].items():
                try:
                    away_team_pos_avrg['ypos'][key] /= away_team_pos_count[key]
                except ZeroDivisionError:
                    away_team_pos_avrg['ypos'][key] = 0
                
                
            print('\n')
            print(i+1)
            print(i)
            print(home_team_pos_avrg)
            print(away_team_pos_avrg)
            print('\n')
        
            # home team
            for key, value in home_team_pos_avrg['xpos'].items():
                home_team_pos_avrg['xpos'][key] = 0
                
            for key, value in home_team_pos_avrg['ypos'].items():
                home_team_pos_avrg['ypos'][key] = 0
                          
            for key, value in home_team_pos_count.items():
                home_team_pos_count[key] = 0
                          
            
            # away team
            for key, value in away_team_pos_avrg['xpos'].items():
                away_team_pos_avrg['xpos'][key] = 0
                
            for key, value in away_team_pos_avrg['ypos'].items():
                away_team_pos_avrg['ypos'][key] = 0
                          
            for key, value in away_team_pos_count.items():
                away_team_pos_count[key] = 0
            
        
        
        if team_id_t == home_team_id:
            home_team_pos_avrg['xpos'][jersey_number_t] += x_pos
            home_team_pos_avrg['ypos'][jersey_number_t] += y_pos
            if x_pos!=0 or y_pos!=0:
                home_team_pos_count[jersey_number_t] +=1      

        elif team_id_t == away_team_id:
            away_team_pos_avrg['xpos'][jersey_number_t] += x_pos
            away_team_pos_avrg['ypos'][jersey_number_t] += y_pos
            if x_pos!=0 or y_pos!=0:
                away_team_pos_count[jersey_number_t] +=1    

        elif team_id_t == 0:
            if jersey_number_t == referee_id:
                pass
                # referee_pos_t['xpos']
                # referee_pos_t['ypos']
            else:
                pass
                    
        
    



600
599
{'xpos': {1: 18.546224715683483, 2: 0, 5: 51.77448807231141, 6: 42.14365684175066, 7: 66.70173772449547, 8: 68.27062190480986, 9: 0, 10: 58.52001252755645, 11: 0, 13: 56.03972515303383, 14: 0, 15: 0, 17: 0, 18: 0, 20: 0, 23: 72.09995015523448, 29: 0, 32: 50.40562112385441, 70: 0, 88: 55.805773788662805, 94: 68.99526152891909}, 'ypos': {1: 34.380264879793664, 2: 0, 5: 36.66495512037153, 6: 21.820107300794618, 7: 52.41538685098257, 8: 17.403077336154503, 9: 0, 10: 32.58257808862936, 11: 0, 13: 30.72391430468876, 14: 0, 15: 0, 17: 0, 18: 0, 20: 0, 23: 30.73018729680674, 29: 0, 32: 49.81643451594171, 70: 0, 88: 16.20796282983811, 94: 32.27210760825376}}
{'xpos': {1: 0, 3: 76.06386013855058, 5: 0, 7: 0, 8: 58.53817969550552, 9: 45.711344387337704, 11: 61.043986688310376, 14: 63.09323335129578, 17: 0, 18: 0, 20: 0, 22: 0, 23: 69.53675651413735, 24: 71.99450691504158, 25: 95.39215744980645, 28: 73.0326527220782, 30: 67.64078366807493, 35: 0, 70: 0, 77: 0, 90: 59.91973895983509}, 'yp



4200
4199
{'xpos': {1: 12.870355790389237, 2: 0, 5: 41.62599574859719, 6: 32.81901893378203, 7: 50.99926599112026, 8: 47.521506931537786, 9: 0, 10: 44.445767806958536, 11: 0, 13: 49.21079644089423, 14: 0, 15: 0, 17: 0, 18: 0, 20: 0, 23: 55.61913343883261, 29: 0, 32: 35.8399200112232, 70: 0, 88: 36.12151428000948, 94: 50.591853187443405}, 'ypos': {1: 32.46800586041866, 2: 0, 5: 31.944979961840556, 6: 23.045571584736322, 7: 18.939257522766088, 8: 22.60019603591458, 9: 0, 10: 24.77619372615048, 11: 0, 13: 25.869452728177226, 14: 0, 15: 0, 17: 0, 18: 0, 20: 0, 23: 26.991720278338974, 29: 0, 32: 39.38045177553276, 70: 0, 88: 11.229249899954347, 94: 20.739538281153294}}
{'xpos': {1: 0, 3: 54.70260561850454, 5: 0, 7: 54.034923771043744, 8: 54.16125120538738, 9: 35.17639446464647, 11: 49.86353493314707, 14: 43.63813148761733, 17: 47.63554265793979, 18: 0, 20: 0, 22: 0, 23: 63.799649667089, 24: 58.7071745005614, 25: 89.14494684509418, 28: 53.46612287317617, 30: 0, 35: 0, 70: 0, 77: 0, 90: 0},